
# ✨ A Super Simple Improvement Strategy  

This is a handy **trick** to identify which values of \( n \) can be optimized.  

🎯 In short: when \( n = i \), the area occupied by \( i \) Christmas trees must be **no less than** that of all cases where \( i \leq n \leq 200 \).  
Why? Because in the worst case, it can reuse the layout for fewer trees and simply choose **not to place** the extra trees.  

💡 **Important**: This not only directly boosts our score, but also creates a **new layout pattern**. Since our new arrangement even leaves space for an entire tree unused, it provides **much greater room for improvement** — with just a little optimization, we can gain significant points!  

🚀 This opens up huge potential for refinement; even small tweaks can lead to substantial score increases.  


In [1]:
#!pip install shapely numba


In [2]:
import shutil


shutil.copy('/kaggle/input/intergration-of-existing-result-current-best/submission.csv',  '/kaggle/working/submission_ori.csv')

'/kaggle/working/submission_ori.csv'

In [3]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

# Set precision for Decimal (25 is good for contest standards)
getcontext().prec = 25
scale_factor = Decimal("1e18")


class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        # Define the 15 vertices of the tree polygon
        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        
        # Apply rotation and translation to the polygon
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, 
            xoff=float(self.center_x * scale_factor), 
            yoff=float(self.center_y * scale_factor)
        )


def load_configuration_from_df(n: int, df: pd.DataFrame) -> list[ChristmasTree]:
    """
    Loads all trees for a given N from the submission DataFrame.
    """
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        # Remove 's' prefix and convert to string for Decimal constructor
        x = str(row["x"])[1:]
        y = str(row["y"])[1:]
        deg = str(row["deg"])[1:]
        
        # Ensure values are present before passing to ChristmasTree constructor
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
        else:
             # Handle cases where configuration might be incomplete/missing
             pass 
             
    return trees


def get_bounding_box_side(trees: list[ChristmasTree]) -> float:
    """
    Calculates the side length S of the minimum bounding square.
    """
    if not trees:
        return 0.0

    # Collect all exterior points from all tree polygons, scale them back down
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    
    side_length = max(max_x - min_x, max_y - min_y)
    
    return side_length


def get_score(trees: list[ChristmasTree], n: int) -> float:
    """
    Calculates the score (S^2 / N) for a given configuration of trees.
    S is the side length of the minimum bounding square.
    """
    side_length = get_bounding_box_side(trees)
    
    # Score is S^2 / N
    score = side_length**2 / n
    return score


def has_overlap(trees: list[ChristmasTree]) -> bool:
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False

    polygons = [t.polygon for t in trees]
    # Use STRtree for efficient proximity queries (optimizes checking pairs)
    tree_index = STRtree(polygons)

    for i, poly in enumerate(polygons):
        # Query for polygons whose bounding boxes overlap with poly
        # This returns the indices of potential overlaps
        indices = tree_index.query(poly)
        
        for idx in indices:
            # Skip checking the polygon against itself
            if idx == i:
                continue
                
            # Perform the precise intersection check
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                # Overlap found!
                return True
    return False

# ----------------------------------------------------------------------
def score_and_validate_submission(file_path: str, output_path: str, max_n: int = 200) -> dict:
    """
    Reads a submission CSV, calculates the total score, checks for overlaps,
    and validates that bounding box sizes are monotonically non-increasing.
    Saves optimized submission to output_path.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {"status": "FAILED", "error": "File Not Found"}
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return {"status": "FAILED", "error": f"CSV Read Error: {e}"}

    total_score = 0.0
    failed_overlap_n = []
    suboptimal_n = []
    
    # Store configurations and their bounding box sizes
    configs = {}
    box_sizes = {}
    original_ids = {}  # 新增：存储原始id
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")

    # First pass: load all configurations and calculate bounding box sizes
    for n in range(1, max_n + 1):
        group_data = df[df["id"].str.startswith(f"{n:03d}_")]
        trees = []
        ids = []  # 新增：记录id列表
        
        for _, row in group_data.iterrows():
            # Remove 's' prefix and convert to string for Decimal constructor
            x = str(row["x"])[1:]
            y = str(row["y"])[1:]
            deg = str(row["deg"])[1:]
            
            # Ensure values are present before passing to ChristmasTree constructor
            if x and y and deg:
                trees.append(ChristmasTree(x, y, deg))
                ids.append(row["id"])  # 新增：保存原始id
        
        if trees:
            configs[n] = trees
            original_ids[n] = ids  # 新增：保存id列表
            box_sizes[n] = get_bounding_box_side(trees)
    
    # Second pass: check for suboptimal configurations
    optimized_df_rows = []
    
    for n in range(1, max_n + 1):
        if n not in configs:
            continue
            
        current_box_size = box_sizes[n]
        current_trees = configs[n]
        current_ids = original_ids[n]  # 新增：获取当前的id列表
        
        # Check if any configuration with more trees has smaller or equal box size
        best_larger_n = None
        best_larger_box_size = float('inf')
        
        for larger_n in range(n + 1, max_n + 1):
            if larger_n in box_sizes:
                if box_sizes[larger_n] <= current_box_size:
                    if box_sizes[larger_n] < best_larger_box_size:
                        best_larger_box_size = box_sizes[larger_n]
                        best_larger_n = larger_n
        
        # If found a better configuration, use it (take first n trees)
        if best_larger_n is not None:
            suboptimal_n.append(n)
            print(f"  ⚠️  N={n:03d}: Suboptimal! Box size {current_box_size:.6f} >= N={best_larger_n} box size {best_larger_box_size:.6f}")
            print(f"      Copying first {n} trees from N={best_larger_n} configuration")
            
            # Use first n trees from the better configuration
            better_trees = configs[best_larger_n][:n]
            configs[n] = better_trees
            box_sizes[n] = get_bounding_box_side(better_trees)
            # 注意：ids保持不变，使用current_ids
        else:
            # 如果没有替换，使用原始配置
            pass
        
        # Now process the (possibly updated) configuration
        trees = configs[n]
        current_score = get_score(trees, n)
        total_score += current_score

        if has_overlap(trees):
            failed_overlap_n.append(n)
            print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.6f})")
        
        # Add to output dataframe - 使用原始id
        for i, tree in enumerate(trees):
            tree_id = current_ids[i] if i < len(current_ids) else f'{n:03d}_{i:03d}'  # 使用原始id
            optimized_df_rows.append({
                'id': tree_id,  # 修改：使用原始id
                'x': f's{tree.center_x}',
                'y': f's{tree.center_y}',
                'deg': f's{tree.angle}'
            })
    
    # Save optimized submission
    optimized_df = pd.DataFrame(optimized_df_rows)
    optimized_df.to_csv(output_path, index=False)
    print(f"\n✅ Optimized submission saved to: {output_path}")
    
    print("\n--- Summary ---")
    if suboptimal_n:
        print(f"⚠️  **Suboptimal configurations found in N: {suboptimal_n}**")
        print(f"    These have been optimized by copying from larger N configurations")
    
    if failed_overlap_n:
        print(f"❌ **Validation FAILED**: Overlaps found in N: {failed_overlap_n}")
        status = "FAILED (Overlaps)"
    else:
        print("✅ **Validation SUCCESSFUL**: No overlaps detected.")
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N): {total_score:.6f}**")
    
    return {
        "status": status,
        "total_score": total_score,
        "failed_overlap_n": failed_overlap_n,
        "suboptimal_n": suboptimal_n,
        "output_file": output_path
    }

# Example usage
input_path = '/kaggle/working/submission_ori.csv'
output_path = '/kaggle/working/submission.csv'

result = score_and_validate_submission(input_path, output_path, max_n=200)
print(result)

--- Scoring and Validation: /kaggle/working/submission_ori.csv (N=1 to 200) ---
  ⚠️  N=102: Suboptimal! Box size 5.953538 >= N=104 box size 5.953536
      Copying first 102 trees from N=104 configuration
  ⚠️  N=103: Suboptimal! Box size 5.953538 >= N=104 box size 5.953536
      Copying first 103 trees from N=104 configuration

✅ Optimized submission saved to: /kaggle/working/submission.csv

--- Summary ---
⚠️  **Suboptimal configurations found in N: [102, 103]**
    These have been optimized by copying from larger N configurations
✅ **Validation SUCCESSFUL**: No overlaps detected.
**Total Submission Score (Σ S²/N): 70.836954**
{'status': 'SUCCESS', 'total_score': np.float64(70.83695423768978), 'failed_overlap_n': [], 'suboptimal_n': [102, 103], 'output_file': '/kaggle/working/submission.csv'}
